In [ ]:
# generally useful imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy.stats as  sc

import geopandas as gpd
from matplotlib.collections import PatchCollection

# For interactive plots
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Introduction

In our analysis we use multiple Datasets of the **Food and Agriculture Organization of the United Nations** (Faostat). 
We start by exploring the Production Livestock Dataset which gives us an broader view of the *Beehives Population* in the world and more fine grained in different continents and countries. 
The following cell describes the research question we plan to solve during our analysis. 

**Research questions**
- How is the beehives population distributed throughout the world?
- How is the population changing over time?
- Influence of reduction/growth of beehives population?
- Does the reduction/growth of beehives affect other factors?
- Can we find patterns why beehives reduce/grow in population?
- Will the bees survive?

# Data Analysis

### Load the data

We simply start our analysis by loading the *Live Animals* dataset (called Production Livestock), which shows us the annual number of different animals between 1961 and 2017. Since we only care about beehives we have to filter our dataset according to this item. Within the group of animals are also beehives, which we us as a value of the total number of bees. Faostat defines beehives to be "an artificial habitation for bees".  



In [ ]:
#load the data
livestock = pd.read_csv('data/Production_Livestock_E_All_Data_(Normalized).csv',  encoding='iso-8859-1')

In [ ]:
livestock.Item.unique()

In [ ]:
beehives = livestock.loc[livestock['Item'] == 'Beehives']
beehives.head()

## We further clean our Dataset

During this process we are cleaning our dataset in order to be able to use throughout our analysis. 
We first observed the "Area" field not to be consistent and needed to split the dataset into smaller subcategories (i.e. countries, continent).

In [ ]:
beehives.Area.unique()

This looks like a lot of countries.
We can further split the countries and see that they actually have countries, continents, regions, consolidations and areas are already reasonably grouped within the data. 
We further explore that some of the countries do not exist anymore. We will take this as given. We could observe that USSR as well as Yugoslavia splitted in 1992. We can use this fact to have a look at the last 25 Years of bees. 

In [ ]:
countries = ['Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia','Australia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium','Belgium-Luxembourg', 'Belize', 'Bolivia (Plurinational State of)','Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Burundi','Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile','China', 'Colombia', 'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba','Cyprus', 'Czechia', 'Czechoslovakia', 'Dominican Republic','Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Ethiopia','Ethiopia PDR', 'Fiji', 'Finland', 'France', 'French Polynesia','Georgia', 'Germany', 'Greece', 'Greenland', 'Guadeloupe', 'Guam','Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti','Honduras', 'Hungary', 'India', 'Iran (Islamic Republic of)','Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kenya','Kyrgyzstan', 'Latvia', 'Lebanon', 'Libya', 'Liechtenstein','Lithuania', 'Luxembourg', 'Madagascar', 'Mali', 'Martinique','Mexico', 'Mongolia', 'Montenegro', 'Morocco', 'Mozambique','Myanmar', 'Netherlands', 'New Caledonia', 'New Zealand','Nigeria', 'Niue', 'Occupied Palestinian Territory', 'Oman','Pakistan', 'Paraguay', 'Poland', 'Portugal', 'Puerto Rico','Republic of Korea', 'Republic of Moldova', 'Romania','Russian Federation', 'Rwanda', 'Samoa', 'Senegal', 'Serbia','Serbia and Montenegro', 'Slovakia', 'Slovenia', 'South Africa','Spain', 'Sudan', 'Sudan (former)', 'Sweden', 'Switzerland','Syrian Arab Republic', 'Tajikistan','The former Yugoslav Republic of Macedonia', 'Timor-Leste','Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Tuvalu','Uganda', 'Ukraine', 'United Kingdom','United Republic of Tanzania', 'United States of America','Uruguay', 'USSR', 'Uzbekistan','Venezuela (Bolivarian Republic of)', 'Viet Nam','Wallis and Futuna Islands', 'Yemen', 'Yugoslav SFR', 'Zambia']
world = ['World']
continents = ['Africa','Americas', 'Asia',  'Europe', 'Oceania']
regions = ['Eastern Africa', 'Middle Africa', 'Northern Africa', 'Southern Africa', 'Western Africa', 'Northern America', 'South America', 'Central America', 'Caribbean','Central Asia', 'Eastern Asia', 'Southern Asia', 'South-Eastern Asia', 'Western Asia', 'Eastern Europe', 'Northern Europe', 'Southern Europe', 'Western Europe', 'Melanesia', 'Micronesia', 'Polynesia', 'Australia & New Zealand']
consolidations = ['European Union', 'China, mainland', 'China, Taiwan Province of']
index_based = ['Least Developed Countries', 'Land Locked Developing Countries','Small Island Developing States','Low Income Food Deficit Countries', 'Net Food Importing Developing Countries']

In [ ]:
beehives_countries = beehives[beehives['Area'].isin(countries)]
beehives_world = beehives.loc[beehives['Area'] == 'World']
beehives_continents = beehives[beehives['Area'].isin(continents)]
beehives_regions = beehives[beehives['Area'].isin(regions)]
beehives_consolidations = beehives.loc[beehives['Area'] == 'European Union']
beehives_index_based = beehives[beehives['Area'].isin(index_based)]

## Beehives population over time and area (Let us plot the world)

Luckily we have a world column which is the sum of the number of beehives over all countries. 
So we first simply plot the world Distribution.  
Like this we can start answering our first two research questions how is the bees population changing over time. Further analysis helped us answering how the bee population is distributed. 

First a world plot.

In [ ]:
plt.plot(beehives_world['Year'], beehives_world['Value'])
plt.title('No of Beehives in the world per year')
plt.xlabel('Years')
plt.ylabel('No of Beehives')
plt.xlim(1960, 2020)

This plot gives us some useful information! This was not what we expected.   
We observe an increasing number of beehives. But wasn't everyone saying bees are dying?  
We can further notice a drop of beehives between 1991 and 2000. Where does this drop come from? 

## Oh we actually see the number of bees are increasing!

By how much is the number increasing? 

In [ ]:
firstYear = int(beehives_world[beehives_world['Year'] == 1961]['Value'])
lastYear = int(beehives_world[beehives_world['Year'] == 2017]['Value'])
lastYear/firstYear

In fact we have an increase of 185.06% over the last 56 Years. 

Looking only at the world between 1990 and 2000 we actually see a drop. We ellaborate further on this. 

In [ ]:
beehives_world[beehives_world['Year'].isin(range(1990,2001))]

In [ ]:
(1-(65933672.0/69952217.0))*100 #taking into account the local maxima year 1991 and the local minima year 1997.

In [ ]:
69952217-65933672

Looking only at these 10 Years (1990-2000) we se the population droped by a number of 4.018.545. Which is a drop of 5.74%
Why is that? 

## Let us try to figure out what happened in between 1990 and 2000. We know there was war in Yugslavia (Bosnia and Herzegovina,  Croatia, Kosovo,  Montenegro,  North Macedonia,  Serbia, Slovenia).

War could be one reason. 

In [ ]:
beehives_yugoslav = beehives_countries.loc[beehives_countries['Area'] == 'Yugoslav SFR']

In [ ]:
beehives_yugoslav[beehives_yugoslav['Year'].isin(range(1990,2001))]

Here we only observe data until 1991. Which is because the countries in Yugoslavia announced independence. We will now look into the splitted countries to see what happened after.

In [ ]:
beehives_countries.loc[beehives_countries['Area'] == 'Croatia']
beehives_countries.loc[beehives_countries['Area'] == 'Bosnia and Herzegovina']
beehives_countries.loc[beehives_countries['Area'] == 'Slovenia']
beehives_countries.loc[beehives_countries['Area'] == 'The former Yugoslav Republic of Macedonia']
beehives_countries.loc[beehives_countries['Area'] == 'Serbia and Montenegro']

In [ ]:
for year in range(1992, 2001):
    croatia = int(beehives_countries.loc[(beehives_countries['Area'] == 'Croatia') & (beehives_countries['Year'] == year)]['Value'])
    bosnia = int(beehives_countries.loc[(beehives_countries['Area'] == 'Bosnia and Herzegovina') & (beehives_countries['Year'] == year)]['Value'])
    slovenia = int(beehives_countries.loc[(beehives_countries['Area'] == 'Slovenia') & (beehives_countries['Year'] == year)]['Value'])
    macedonia = int(beehives_countries.loc[(beehives_countries['Area'] == 'The former Yugoslav Republic of Macedonia') & (beehives_countries['Year'] == year)]['Value'])
    sm = int(beehives_countries.loc[(beehives_countries['Area'] == 'Serbia and Montenegro') & (beehives_countries['Year'] == year)]['Value'])
    print("Year: %s, No of Beehives: %s" % (year , croatia+bosnia+slovenia+macedonia+sm))

Looking at our data we see that this cannot be the reason and that war is not killing bees and even increase. Since the values stayed pretty stable. We will look into countries having greater drop between 1992 and 1995.

So in which other countries do bees decrease?

## Decreasing Countries 1992-1997

Let us now dive deeper into countries.  
We try to figure out countries that experienced a significant drop of bees between 1992 and 1997.

In [ ]:
count_decreasingNumber = 0
big_decrease = []
for country in countries:
    try:
        sm1 = int(beehives_countries.loc[(beehives_countries['Area'] == country) & (beehives_countries['Year'] == 1992)]['Value'])
        sm2 = int(beehives_countries.loc[(beehives_countries['Area'] == country) & (beehives_countries['Year'] == 1997)]['Value'])
        if sm2/sm1 < 1:
            count_decreasingNumber += 1
        #if sm2-sm1<-100000:
        if ((1-(sm2/sm1))*100)>66 or sm2-sm1<-100000:
            print(country)
            print("Ratio 1997/1992: -%.2f%%" % ((1-(sm2/sm1))*100))
            print("Number of Beehives in 1992: %d" % sm1)
            print("Number of Beehives in 1997: %d" % sm2)
            print("------------------")
            big_decrease.append(country)
    except:
        continue
print("The number of beehives in the period from 1992-1997 decreased in %s of %s countries." % (count_decreasingNumber, len(countries)))

A significant decrease >100.000 or over 66% beehives can be noticed in the following countries.

In [ ]:
print(*big_decrease, sep = ", ")  

So now let us plot these countries. Are they seriously decreasing. 

In [ ]:
@interact
def show_beehives_for_country(country=big_decrease):
    beehives_country=beehives_countries.loc[beehives_countries['Area']==country]
    
    plt.plot(beehives_country['Year'],beehives_country['Value'],'-*')
    plt.title('Country: '+country)
    plt.xlabel('Years')
    plt.ylabel('Number')
    plt.xlim(1960, 2020)

Looking at these plots we see that Bulgaria, China and Romania recovered quickly after the decline, while Egypt, Germany, Poland, Romania and the United States of America dropped the numbers continously.

Here we could talk about the number that each country reduced.
Some of them increased even over 65 percent. 

## Is this trend continuing up to the year 2017?

In [ ]:
count_decreasingNumber2017 = 0
big_decrease2017 = []
for country in countries:
    try:
        sm1 = int(beehives_countries.loc[(beehives_countries['Area'] == country) & (beehives_countries['Year'] == 1992)]['Value'])
        sm2 = int(beehives_countries.loc[(beehives_countries['Area'] == country) & (beehives_countries['Year'] == 2017)]['Value'])
        if sm2/sm1 < 1:
            count_decreasingNumber2017 += 1
        if ((1-(sm2/sm1))*100)>66 or sm2-sm1<-100000:
            print(country)
            print("Ratio 2017/1992: -%.2f%%" % ((1-(sm2/sm1))*100))
            print("Number of Beehives in 1992: %d" % sm1)
            print("Number of Beehives in 2017: %d" % sm2)
            print("------------------")
            big_decrease2017.append(country)
    except:
        continue
print("The number of beehives in the period from 1992-2017 decreased in %s of %s countries." % (count_decreasingNumber2017, len(countries)))

Looking at our data we see now less countries decreasing. 19 compared to 26. But the ratio for some european countries increased a lot and switzerland appeared as a newcomer with a loss of 38.21%.

Now let us plot these again.

In [ ]:
@interact
def show_beehives_for_country(country=big_decrease2017):
    beehives_country=beehives_countries.loc[beehives_countries['Area']==country]
    
    plt.plot(beehives_country['Year'],beehives_country['Value'],'-*')
    plt.title('Country: '+country)
    plt.xlabel('Years')
    plt.ylabel('Number')
    plt.xlim(1992, 2017)

In [ ]:
print(*big_decrease2017, sep = ", ")  

These are the countries with a big decrease again. We notice that for the longterm trend the number of countries having a high percentage loss decreased. Mostly bigger countries struggle by loosing a high absolute  value of beehives. But looking into the percentage difference only Italy occurs loosing a higher percentage over 66%.

In [ ]:
beehives_bigDecrease = beehives[beehives['Area'].isin(big_decrease2017)]

In [ ]:
beehives_bigDecrease

We can see the trend is continuing until 2017.  

## And what about increasing countries?

In [ ]:
count_increasingNumber = 0
big_increase = []
for country in countries:
    try:
        sm1 = int(beehives_countries.loc[(beehives_countries['Area'] == country) & (beehives_countries['Year'] == 1992)]['Value'])
        sm2 = int(beehives_countries.loc[(beehives_countries['Area'] == country) & (beehives_countries['Year'] == 2017)]['Value'])
        if sm2/sm1 > 1:
            count_increasingNumber += 1
#         if sm2-sm1>100000:
        if ((sm2/sm1)*100)>300 or sm2-sm1>200000:

            print(country)
            print("Ratio 2017/1992: %.2f%%" % ((sm2/sm1)*100))
            print("Number of Beehives in 1992: %d" % sm1)
            print("Number of Beehives in 2017: %d" % sm2)
            print("------------------")
            big_increase.append(country)
    except:
        continue
print("The number of beehives in the period from 1992-2017 increased in %s of %s countries." % (count_increasingNumber, len(countries)))

Notice: There is a difference in the total number countries. This is due to newcomers who were not considered a country in 1992. For example 'Serbia and Montenegro' split to 'Serbia' and 'Montenegro'.  

In [ ]:
@interact
def show_beehives_for_country(country=big_increase):
    beehives_country=beehives_countries.loc[beehives_countries['Area']==country]
    
    plt.plot(beehives_country['Year'],beehives_country['Value'],'-*')
    plt.title('Country: '+country)
    plt.xlabel('Years')
    plt.ylabel('Number')
    plt.xlim(1992, 2020)

Looking at these plots we can clearly observe a lot of countries having a growing bee population. More than countries decreasing.  
We can also observe the countries are not specific to one continent but rather include different continents and areas. 


## Let us plot the continents

Let us no take a step back and look from a higher perspective on continents.  

We saw bees are increasing around the world. Is this also true for every continent?

We first draw a stacked boxplot and continue to look at each country in an interactive plot. 

In [ ]:
beehives_continents.loc[beehives_continents['Year'] == 2017].sort_values('Value', ascending=False)

In [ ]:
N = len(beehives_continents[beehives_continents['Area'] == 'Africa']['Value'])
africa = np.array(beehives_continents[beehives_continents['Area'] == 'Africa']['Value'])
europe = np.array(beehives_continents[beehives_continents['Area'] == 'Europe']['Value'])
america = np.array(beehives_continents[beehives_continents['Area'] == 'Americas']['Value'])
oceania = np.array(beehives_continents[beehives_continents['Area'] == 'Oceania']['Value'])
asia = np.array(beehives_continents[beehives_continents['Area'] == 'Asia']['Value'])

ind = np.arange(N)    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

plt.figure(figsize=(20,10))

p1 = plt.bar(ind, asia, width)
p2 = plt.bar(ind, europe, width, bottom=asia)
p3 = plt.bar(ind, africa, width, bottom=europe+asia)
p4 = plt.bar(ind, america, width, bottom=asia+europe+africa)
p5 = plt.bar(ind, oceania, width, bottom=asia+america+europe+africa)
plt.ylabel('Number of Beehives')
plt.xlabel('Year')
plt.title('Distribution of Beehives in Europe')
plt.xticks(ind, (range(1961, 2018)), rotation=45)
plt.legend((p1[0], p2[0], p3[0], p4[0], p5[0]), ('Asia', 'Europe', 'Africa', 'America', 'Oceania'))

plt.show()

The plot reveals a lot about the distribution. We can observe that Asia on top is the biggest contributor of beehives, followed by a declining europe and an increasing Africa. America seems to be very stable while oceania is only responsible for a fairly small amount of beehives.  
We further look into the curves more in detail. We also see a big drop in europe. Which could be responsible for the sudden drop in the world.  
In fact Europe was the continent with the biggest bee population before the drop.  

We will look more deeply into the continents change ove time by plotting their distribution individually. 

In [ ]:
@interact
def show_beehives_for_country(continent=continents):
    beehives_continent=beehives_continents.loc[beehives_continents['Area']==continent]
    
    plt.plot(beehives_continent['Year'],beehives_continent['Value'],'-*')
    plt.title('Country: '+continent)
    plt.xlabel('Years')
    plt.ylabel('Number')
    plt.xlim(1960, 2020)

Looking at the plots individually we observe **Africa** is increasing it's beehives population as well as **Asia**. **Oceania** has a huge increase of beehives from 2012 to 2013, and from 2016 to 2017, we expect more to come.  
**America** and **Europe** have an unproportional declining population between 1990 and 2000. America recovered again while Europe still lacks behind it's pre 1990 value. 


In [ ]:
# beehives_continents.loc[(beehives_continents['Area'] == 'Europe') & ~(beehives_continents['Year'] < 1985) & ~(beehives_continents['Year'] > 1998)]

## Conclusion

Looking at the data we see that countries that have a lot of bees are increasing there effort in growing the population. While other western countries (Germany, Italy, France, Switzerland, Austria and USA) find themselves in a situation of decreasing population.  

## Let us now also take a quick look on the 10 countries with the biggest beehives population in 2017

In [ ]:
beehives_countries[beehives_countries['Year'] == 2017].sort_values('Value', ascending=False)[['Area','Value']].head(10)

Looking at the top ten we can see that they are spread around the world. With this we mean there are contributors in Asia, Europe, America and Africa. Only Oceania is missing. 

In [ ]:
topTenContributors = beehives_countries[beehives_countries['Year'] == 2017].sort_values('Value', ascending=False)['Value'].head(10).sum()
worldPopulationBeehives = int(beehives_world[beehives_world['Year'] == 2017].Value)
ratioTopTenWorld = topTenContributors/worldPopulationBeehives
print("Here we can observe that our top ten contributors are responsible for %s%% of the worlds population of beehives, which is %s in total" % (ratioTopTenWorld, worldPopulationBeehives))

Let us now give some contribution to the smaller countries which support the beehives population as well. Therefore we add a column that is dividing the total number of beehives in the country divided by the estimate population. 

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

Let us just check our format.

In [ ]:
world.head()

We saw some differently spelled data, which we corrected in order to have correct the same names of countries.

In [ ]:
beehives_perPopulation = beehives_countries.copy() 
beehives_perPopulation = beehives_perPopulation.replace({'Russian Federation': 'Russia', 'Bosnia and Herzegovina': 'Bosnia and Herz.', 'Belgium-Luxembourg': 'Luxembourg', 'Bolivia (Plurinational State of)':'Bolivia', 'Central African Republic':'Central African Rep.', 'China, Taiwan Province of': 'Taiwan', 'Dominican Republic': 'Dominican Rep.', 'Ethiopia PDR': 'Ethiopia', 'Iran (Islamic Republic of)':'Iran', 'Syrian Arab Republic':'Syria', 'North Macedonia':'Macedonia', 'Venezuela (Bolivarian Republic of)': 'Venezuela', 'Viet Nam':'Vietnam', 'Czechoslovakia':'Czechia'})
beehives_perPopulation = world.set_index('name').join(beehives_perPopulation.set_index('Area'))

In [ ]:
beehives_perPopulation['Per_Population'] = beehives_perPopulation['Value']/beehives_perPopulation['pop_est']

In [ ]:
beehives_perPopulation['Area'] = beehives_perPopulation.index

In [ ]:
beehives_perPopulation[beehives_perPopulation['Year'] == 2017].sort_values('Per_Population', ascending=False)[['Value']].head(10)

Now we see different countries and there is New Zealand. One of the Oceanian countries. Interestingly 7 out of 10 countries are considered to be in eastern europe. 

## EU27 Distribution

In [ ]:
eu27 = ['Austria', 'Italy', 'Belgium', 'Latvia', 'Bulgaria', 'Lithuania', 'Croatia', 'Luxembourg', 'Cyprus', 'Malta', 'Czechia', 'Netherlands', 'Denmark', 'Poland', 'Estonia', 'Portugal', 'Finland', 'Romania', 'France', 'Slovakia', 'Germany', 'Slovenia', 'Greece', 'Spain', 'Hungary', 'Sweden', 'Ireland', 'United Kingdom']

beehives_eu27 = beehives[beehives['Area'].isin(eu27)]
beehives_eu27_2017 = beehives_eu27.loc[beehives_eu27['Year']==2017]
beehives_eu27_2017 = beehives_eu27_2017.dropna(subset=['Value'])



In [ ]:
plt.figure(figsize=(15,8))
plt.bar(beehives_eu27_2017['Area'], beehives_eu27_2017['Value'])
plt.title('Bees Population EU27')
plt.xlabel('Number of Beehives')
plt.xticks(rotation=60);

Looking at the dataset we observe Spain, Greece and Poland to have the three biggest bees population. 

## Let us now for aesthetic reasons plot a map 

There we can reuse our beehives_perPopulation DataFrame

In [ ]:
years = beehives_perPopulation.Year.unique()
years = years[~np.isnan(years)]
years = beehives_perPopulation.Year.unique()
years = years[~np.isnan(years)]
beehives_perPopulation.Year = beehives_perPopulation.Year.fillna(0)
beehives_perPopulation.Year = beehives_perPopulation.Year.astype(int)
beehives_perPopulation.Value = beehives_perPopulation.Value.fillna(0)

In [ ]:
beehives_perPopulation.Year.loc[beehives_perPopulation.Year == 0]
newDF = beehives_perPopulation.loc[beehives_perPopulation['Year'] == 0].copy()
for year in years: 
    newDF.Year = year
    beehives_perPopulation = beehives_perPopulation.append(newDF, ignore_index = True)

In [ ]:
mapPlot_beehives = pd.DataFrame({})

perc =[.10, .20,.30,.40,.50,.60,.70,.80,.90] 
for year in years:
    beehives_splitted_inYears = beehives_perPopulation.loc[beehives_perPopulation['Year']==year].copy()
    beehives_splitted_inYears['categorize value'] = np.where(beehives_splitted_inYears['Value']==0, 0, \
                                            np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['10%'], 1, \
                                                np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['20%'], 2, \
                                                         np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['30%'], 3, \
                                                                  np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['40%'], 4, \
                                                                           np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['50%'], 5, \
                                                                                    np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['60%'], 6, \
                                                                                             np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['70%'], 7, \
                                                                                                      np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['80%'], 8, \
                                                                                                            np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['90%'], 9, 10))))))))))
    beehives_splitted_inYears['value percentile'] = np.where(beehives_splitted_inYears['Value']==0, 0, \
                                            np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['10%'], beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['10%'], \
                                                np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['20%'], beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['20%'], \
                                                         np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['30%'], beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['30%'], \
                                                                  np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['40%'], beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['40%'], \
                                                                           np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['50%'], beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['50%'], \
                                                                                    np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['60%'], beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['60%'], \
                                                                                             np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['70%'], beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['70%'], \
                                                                                                      np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['80%'], beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['80%'], \
                                                                                                            np.where(beehives_splitted_inYears['Value']<=beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['90%'], beehives_splitted_inYears.mask(beehives_splitted_inYears.Value == 0).Value.describe(percentiles = perc)['90%'], beehives_splitted_inYears.Value.max()))))))))))
    mapPlot_beehives = mapPlot_beehives.append(beehives_splitted_inYears, ignore_index = True)

In [ ]:
years = range(1992,2018)
@interact
def show_beehives_for_country(year=years):
    fig, ax = plt.subplots(1, figsize=(20, 12))
    mapPlot_beehives_Year=mapPlot_beehives.loc[mapPlot_beehives['Year']==year]

    mapPlot_beehives_Year.plot(column='categorize value',cmap='Blues', ax=ax, linewidth=0.8, edgecolor='0.8')
    ax.axis('off')
    ax.set_title('Beehives population in the world', fontdict={'fontsize': '50', 'fontweight' : '3'})
    vmax = mapPlot_beehives_Year['value percentile'].max()
    sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=0, vmax=vmax))
    #sm._A = []
    #cbar = fig.colorbar(sm)


In [ ]:
mapPlot_beehives.head()

## We found out that pesticides are one of the worst killers for bees. These include Organo-phosphates, Carbamates and Pyrethroids. They are highly toxic.

In [ ]:
pesticides = pd.read_csv('data/Inputs_Pesticides_Use_E_All_Data_(Normalized).csv',  encoding='iso-8859-1')

Let us now first again preprocess the data, filter for our countries and then take a closer look if there is a correlation.

In [ ]:
pesticides.head()

In [ ]:
pesticides.Item.unique()[range(5)]

Oh! Our data looks bad formatted. Let us first fix this.

In [ ]:
pesticides['Item'] = pesticides['Item'].str.replace('\x96', '-')

In [ ]:
pesticides.Item.unique()[range(5)]

Now this looks better.

For us interesting are Organo-phosphates, Carbamates and Pyrethroids as well as the total use in the countries where bees are decreasing.

In [ ]:
pesticides_total = pesticides.loc[(pesticides['Item'] == 'Pesticides (total)') & (pesticides['Area'].isin(big_decrease))]
pesticides_total_world = pesticides.loc[(pesticides['Item'] == 'Pesticides (total)') & (pesticides['Area'] == 'World')]

In [ ]:
pesticides_organoPhosphates = pesticides.loc[(pesticides['Item'] == 'Insecticides - Organo-phosphates') & (pesticides['Area'].isin(big_decrease))]
pesticides_carbamates = pesticides.loc[(pesticides['Item'] == 'Insecticides - Carbamates') & (pesticides['Area'].isin(big_decrease))]
pesticides_pyrethroids = pesticides.loc[(pesticides['Item'] == 'Insecticides - Pyrethroids') & (pesticides['Area'].isin(big_decrease))]

Let us now first see how the use of pesticides across the world is. Unfortunately we only have data from 1990 to 2017.

In [ ]:
plt.plot(pesticides_total_world['Year'], pesticides_total_world['Value'])
plt.title('Use of Pesticides total in the world per year')
plt.xlabel('Years')
plt.ylabel('Use of pesticides in tonnes')
plt.xlim(1990, 2020)

### Here we see an increasing number of pesticides throughout the world. How is the increase considered only the toxic pesticides?

In [ ]:
@interact
def show_beehives_for_country(country=big_decrease2017):
    pesticides_organoPhosphates_country=pesticides_organoPhosphates.loc[pesticides_organoPhosphates['Area']==country]
    plt.plot(pesticides_organoPhosphates_country['Year'],pesticides_organoPhosphates_country['Value'],'-*')
    plt.title('Country: '+country)
    plt.xlabel('Years')
    plt.ylabel('Number')
    plt.xlim(1990, 2020)

In [ ]:
@interact
def show_beehives_for_country(country=big_decrease2017):
    pesticides_carbamates_country=pesticides_carbamates.loc[pesticides_carbamates['Area']==country]
    plt.plot(pesticides_carbamates_country['Year'],pesticides_carbamates_country['Value'],'-*')
    plt.title('Country: '+country)
    plt.xlabel('Years')
    plt.ylabel('Number')
    plt.xlim(1990, 2020)

In [ ]:
@interact
def show_beehives_for_country(country=big_decrease2017):
    pesticides_pyrethroids_country=pesticides_pyrethroids.loc[pesticides_pyrethroids['Area']==country]
    plt.plot(pesticides_pyrethroids_country['Year'],pesticides_pyrethroids_country['Value'],'-*')
    plt.title('Country: '+country)
    plt.xlabel('Years')
    plt.ylabel('Number')
    plt.xlim(1990, 2020)

This data does look hard to obtain since a lot of countries are not covered in this dataset. However some are. From looking at the three different pesticides we see that Pyrethroids are increasing in Austria, France, Germany, France, Italy and Switzerland. The two other pesticides decrease.

In [ ]:
joined = beehives.loc[beehives["Area"].isin(countries)]\
                 .set_index(["Area","Year"])\
                 .join(pesticides_total.loc[pesticides_total["Area"].isin(countries)].set_index(["Area","Year"]), lsuffix='_beehives', rsuffix='_pesticides')
joined = joined.reset_index()
joined = joined[['Area', 'Year', 'Value_beehives', 'Value_pesticides']]

In [ ]:
import seaborn as sns
pearson_corr = joined.corr("pearson")
sns.heatmap(pearson_corr, fmt='.2g',annot=True, cmap="OrRd");

## Looking into pesticides toxic for beehives

In [ ]:
toxic_pesticides = ['Insecticides - Organo-phosphates', 'Insecticides - Carbamates', 'Insecticides - Pyrethroids']
pesticides_toxic = pesticides.loc[(pesticides['Item'].isin(toxic_pesticides)) & (pesticides['Area'].isin(countries))]

In [ ]:
joined = beehives.loc[beehives["Area"].isin(countries)]\
                 .set_index(["Area","Year"])\
                 .join(pesticides_toxic.loc[pesticides_toxic["Area"].isin(countries)].set_index(["Area","Year"]), lsuffix='_beehives', rsuffix='_pesticides_toxic')
joined = joined.reset_index()
joined = joined[['Area', 'Year', 'Value_beehives', 'Value_pesticides_toxic']]

In [ ]:
import seaborn as sns
pearson_corr = joined.corr("pearson")
sns.heatmap(pearson_corr, fmt='.2g',annot=True, cmap="OrRd");

### But what are Pyrethroids and why is the number increasing this much? 

Insecticides in general are a substance to kill insects. Some of them are widely regarded as safe to the human but '2250 times more toxic to insects'. [https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6174339/]. One of those is the Pyrethroid. It is recommendd for 'in-home insect control' to keep the house clean of insects and so widely spread. They can also be found in different products, like mosquito repellents, soap for pets or in our sprinkled in ones garden. [https://www.theverge.com/2014/2/19/5423480/popular-poison-pyrethroid-health-risks].
The effects of pesticides on bees can weaken their immune system which makes them less productive and leads to a decline of bees population.

# An interactive plot of percentage changes during the years

In [ ]:
selected_countries = ['Austria', 'Bulgaria', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Italy', 'Poland', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'Canada', 'Mexico', 'United States of America']

In [ ]:
beehives_selected = beehives.loc[beehives.Area.isin(selected_countries)]

@interact(Starting_Year=(1961, 2016, 1), End_Year=(1962, 2017, 1))
def show_beehives_for_country(Starting_Year=1961, End_Year=2017):
    
    old = np.array(beehives_selected.loc[beehives_selected.Year == Starting_Year, 'Value'])
    new = np.array(beehives_selected.loc[beehives_selected.Year == End_Year, 'Value'])
    
    res = pd.DataFrame((new - old) / old) * 100
    res.index = beehives_selected.Area.unique()
    res.columns = ['Value']
    
    fig = plt.figure(figsize=(15, 8))
    plt.axis([0, len(selected_countries), -150, 500])
    plt.bar(x = res.index, height = res.Value, color=(res['Value'] > 0).map({True: 'b', False: 'r'}))
    plt.xticks(rotation=45)
    plt.axhline(y = 0,linewidth = 1, color = 'k');

Here we have a plot of percentage changes for some selected countries during the years. You can use the sliders to change years between which the percentage change is computed.

# Looking at the correlation of the forest land and beehives  

In [ ]:
beehives_countries

In [ ]:
land_use = pd.read_csv('data/Inputs_LandUse_E_All_Data_(Normalized).csv',  encoding='iso-8859-1')
#land_use_countries = land_use[land_use['Area'].isin(countries)]
#land_use_continents = land_use[land_use['Area'].isin(continents)]

Load datasets

In [ ]:
# Forest land, Country Area, Agricultural land
forest_land = land_use.loc[land_use['Item'] == 'Forest land'].copy()
agricultural_land = land_use.loc[land_use['Item'] == 'Agricultural land'].copy()
country_land = land_use.loc[land_use['Item'] == 'Country area'].copy()
inland_waters = land_use.loc[land_use['Item'] == 'Inland waters'].copy()

forest_land.drop(forest_land[forest_land["Element"] == "Carbon stock in living biomass"].index, inplace = True)

First look at the data

In [ ]:
country_land.head(5)

Calculating ratio for forest land, agricultural land and beehives. These quantities are defined as:

$$ Ratio_{forest} = \frac{ForestArea}{TotalArea} $$

$$ Ratio_{agriculture} = \frac{AgriculturalArea}{TotalArea} $$

$$ Ratio_{beehives} = \frac{BeehivesNumber}{TotalArea} $$

In [ ]:
years = range(1960, 2018)
all_areas = countries + continents + ["World"]

for country in all_areas:
    #print("country: ", country)
    for year in years:
        # calculate and insert ratio in corresponding dataframe(for each country and year)
        try:
            country_elem = country_land.loc[(country_land["Area"] == country) & (country_land["Year"] == year), "Value"].values[0]
            try:
                forest_elem = forest_land.loc[(forest_land["Area"] == country) & (forest_land["Year"] == year), "Value"].values[0]
                forest_ratio = forest_elem / country_elem
                forest_land.loc[(forest_land["Area"] == country) & (forest_land["Year"] == year),"Ratio"] = forest_ratio
            except:
                pass
            try:
                agricultural_elem = agricultural_land.loc[(agricultural_land["Area"] == country) & (agricultural_land["Year"] == year), "Value"].values[0]
                agricultural_ratio = agricultural_elem / country_elem
                agricultural_land.loc[(agricultural_land["Area"] == country) & (agricultural_land["Year"] == year),"Ratio"] = agricultural_ratio
            except:
                pass
            try:
                waters_elem = inland_waters.loc[(inland_waters["Area"] == country) & (inland_waters["Year"] == year), "Value"].values[0]
                waters_ratio = waters_elem / country_elem
                inland_waters.loc[(inland_waters["Area"] == country) & (inland_waters["Year"] == year),"Ratio"] = waters_ratio
            except:
                pass
            try:
                beehives_elem = beehives.loc[(beehives["Area"] == country) & (beehives["Year"] == year), "Value"].values[0]
                beehives_ratio = beehives_elem / country_elem
                beehives.loc[(beehives["Area"] == country) & (beehives["Year"] == year),"Ratio"] = beehives_ratio
            except:
                pass
        except:
            pass

Visualizing correlation between forest ratio and beehives number, in countries and continents.

In [ ]:
@interact
def plot_agricultural_land(area=countries):
    try:
        #agricultural_tmp = agricultural_land[(agricultural_land["Country or Area"] == area) & (agricultural_land["Element"] == "Area")][["Value","Year"]]
        forest_tmp = forest_land[forest_land["Area"] == area][["Ratio","Year"]]
        beehives_tmp = beehives[beehives["Area"] == area][["Value","Year"]]
    
        fig, ax1 = plt.subplots()
        
        color = 'tab:red'
        ax1.set_xlabel('Years')
        ax1.set_ylabel('Beehives number', color=color)
        beehives_tmp.plot(ax=ax1, x="Year", y="Value", color=color, label="Beehives number")
        ax1.tick_params(axis='y', labelcolor=color)

        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

        color = 'tab:blue'
        ax2.set_ylabel('Forest / Total land ratio', color=color)  # we already handled the x-label with ax1
        forest_tmp.plot(ax=ax2, x="Year", y="Ratio", color=color, label="Forest ratio")
        ax2.tick_params(axis='y', labelcolor=color)
        ax1.get_legend().remove()
        ax2.get_legend().remove()
        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        plt.show()
    except:
        pass

## Forests analysis

In [ ]:
forest_land.loc[(forest_land["Year"] == 2017) & (forest_land["Area"] == "World"), ["Area", "Value", "Ratio"]]

This means that ~30% of the world land surface area is covered by forests.
Visualizing forests area per continent. 

Visualizing forests area per continent. 

In [ ]:
forest_land.sort_values(by=["Value"], ascending=False)\
           .loc[(forest_land["Year"] == 2017) & (forest_land["Area"].isin(continents)), ["Area", "Value"]]\
           .plot.bar(x="Area",y="Value", label="Forests surface(Ha)", title="Forests surface area per continent")

In [ ]:
forest_land.sort_values(by=["Ratio"], ascending=False)\
           .loc[(forest_land["Year"] == 2017) & (forest_land["Area"].isin(continents)), ["Area", "Ratio"]]

### Forests analysis per country

Plotting countries with more forests.

In [ ]:
forest_land.sort_values(by=["Value"], ascending=False)\
           .loc[(forest_land["Year"] == 2017) & (forest_land["Area"].isin(countries)), ["Area", "Value"]]\
           .plot.bar(x="Area", y="Value", title="Top 15 countries for forest covered area", label="Forests surface(Ha)", figsize=(16,4))

Plotting the total surface area in each country. we can observe a long tailed distribution.

In [ ]:
country_land.sort_values(by=["Value"], ascending=False)\
           .loc[(country_land["Year"] == 2017) & (country_land["Area"].isin(countries)), ["Area", "Value"]]\
           .head(10)\
           .plot.barh(x="Area", y="Value", title="Total surface area in a country", label="Country surface(Ha)")

It seems that Russia has almost the double of forests than any other country, but also the double in total area.
We proceed by analysing the ratio between the area covered by forests and the total available surface.

In [ ]:
forest_land.sort_values(by=["Ratio"], ascending=False)\
           .loc[(forest_land["Year"] == 2017) & (forest_land["Area"].isin(countries)), ["Area","Ratio"]]\
           .head(10)

In [ ]:
forest_land.sort_values(by=["Ratio"], ascending=False)\
           .loc[(forest_land["Year"] == 2017) & (forest_land["Area"].isin(countries)), ["Area", "Ratio"]]\
           .plot.bar(x="Area", y="Ratio", title="Forest ratio per country", label="Forest ratio", figsize=(16,4))

### Discover trends in number of bees

In [ ]:
forest_land.sort_values(by=["Ratio"], ascending=False)\
           .loc[(forest_land["Year"] == 2017) & (forest_land["Area"].isin(big_decrease2017)), ["Area","Ratio"]]\

In [ ]:
forest_land.sort_values(by=["Ratio"], ascending=False)\
            .loc[(forest_land["Year"] == 2017) & (forest_land["Area"].isin(big_increase)), ["Area","Ratio"]]\
            .head(10)

This result doesn't confirm the fact highlighted before, but could it still be some correlation? Let's check Spearman and Pearson correlations.

In [ ]:
joined = beehives.loc[beehives["Area"].isin(countries)]\
                 .set_index(["Area","Year"])\
                 .join(forest_land.loc[forest_land["Area"].isin(countries)].set_index(["Area","Year"]), lsuffix='_beehives', rsuffix='_forest')
joined = joined.reset_index()

In [ ]:
joined = joined[['Area', 'Year', 'Value_beehives', 'Ratio_beehives', 'Value_forest', 'Ratio_forest']]

##### Pearson correlation

In [ ]:
import seaborn as sns
pearson_corr = joined.corr("pearson")
sns.heatmap(pearson_corr, fmt='.2g',annot=True, cmap="OrRd");

We can see that we have some week correlation between forests surface area in a country and number of beehives in that country. 

The forest concentration is probably not the only factor that influence the number of beehives, but it could have an impact on it. 

Let's calculate the Pearson correlation for other factors that could have influence too, like agricultural land area and inland waters surface.

##### Analysis of agriculture and beehives correlation

In [ ]:
joined = beehives.loc[beehives["Area"].isin(countries)]\
                 .set_index(["Area","Year"])\
                 .join(agricultural_land.loc[agricultural_land["Area"].isin(countries)].set_index(["Area","Year"]), lsuffix='_beehives', rsuffix='_agriculture')
joined = joined.reset_index()

joined = joined[['Area', 'Year', 'Value_beehives', 'Ratio_beehives', 'Value_agriculture', 'Ratio_agriculture']]
spearman_corr = joined.corr("pearson")
sns.heatmap(spearman_corr, fmt='.2g',annot=True, cmap="OrRd");

It seems that we have a good correlation between the agricultural surface area and the number of beehives.

##### Analysis of inland water and beehives correlation

In [ ]:
joined = beehives.loc[beehives["Area"].isin(countries)]\
                 .set_index(["Area","Year"])\
                 .join(inland_waters.loc[inland_waters["Area"].isin(countries)].set_index(["Area","Year"]), lsuffix='_beehives', rsuffix='_water')
joined = joined.reset_index()

joined = joined[['Area', 'Year', 'Value_beehives', 'Ratio_beehives', 'Value_water', 'Ratio_water']]
spearman_corr = joined.corr("pearson")
sns.heatmap(spearman_corr, fmt='.2g',annot=True, cmap="OrRd");

It seems that also here we have a good correlation between the inland water covered surface and the number of beehives.

### We feel like we need some data about colony losses, which this dataset doesn't provide

Colony loss disorder is an importan topic blabla...  
We found data for the US provided by USDA: https://www.nass.usda.gov/Surveys/Guide_to_NASS_Surveys/Bee_and_Honey/  
The data is poorly formatted so we will have to do a lot of preprocessing

In [ ]:
import csv

header = [['State', 'Colonies', 'Maximum', 'Lost', 'Percent lost', 'Added', 'Renovated', 'Percent renovated'],
            ['State', 'Mites', 'Parasites', 'Diseases', 'Pesticides', 'Other', 'Unknown']]

def create_dataframe(path, index):
    
    title = []
    data = []
    
    with open(path, encoding="utf8", errors='ignore') as input_file:
        reader = csv.reader(input_file)
        
        for cnt, row in enumerate(reader):
            
            if row[1] == 't':
                title.append(row)
                
            if row[1] == 'd':
                data.append(row[2:])

    date = title[1][-1].split('-')[-1]
    
    df = pd.DataFrame(data)
    df.columns = header[index]
    df['Date'] = pd.to_datetime(date)
    return df[df.State != '']

In [ ]:
folder = 'data/Bees/'

files = ['BeeColonies-05-12-2016/hcny_p01_t005.csv', 'BeeColonies-05-12-2016/hcny_p02_t001.csv',
        'BeeColonies-05-12-2016/hcny_p03_t007.csv', 'BeeColonies-05-12-2016/hcny_p04_t008.csv',
         
        'BeeColonies-08-01-2017/hcny_p04_t005.csv', 'BeeColonies-08-01-2017/hcny_p05_t001.csv',
        'BeeColonies-08-01-2017/hcny_p06_t007.csv', 'BeeColonies-08-01-2017/hcny_p07_t008.csv',
         
        'BeeColonies-08-01-2018/hcny_p04_t005.csv', 'BeeColonies-08-01-2018/hcny_p05_t001.csv',
        'BeeColonies-08-01-2018/hcny_p06_t007.csv', 'BeeColonies-08-01-2018/hcny_p07_t008.csv',
         
        'BeeColonies-08-01-2019/hcny_p03_t005.csv', 'BeeColonies-08-01-2019/hcny_p04_t001.csv',
        'BeeColonies-08-01-2019/hcny_p05_t007.csv', 'BeeColonies-08-01-2019/hcny_p06_t008.csv',
        'BeeColonies-08-01-2019/hcny_p07_t011.csv']

files2 = ['BeeColonies-05-12-2016/hcny_p06_t002.csv', 'BeeColonies-05-12-2016/hcny_p07_t013.csv',
        'BeeColonies-05-12-2016/hcny_p08_t009.csv', 'BeeColonies-05-12-2016/hcny_p09_t010.csv',
         
        'BeeColonies-08-01-2017/hcny_p10_t002.csv', 'BeeColonies-08-01-2017/hcny_p11_t013.csv',
        'BeeColonies-08-01-2017/hcny_p12_t009.csv', 'BeeColonies-08-01-2017/hcny_p13_t010.csv',
         
        'BeeColonies-08-01-2018/hcny_p10_t002.csv', 'BeeColonies-08-01-2018/hcny_p11_t013.csv',
        'BeeColonies-08-01-2018/hcny_p12_t009.csv', 'BeeColonies-08-01-2018/hcny_p13_t010.csv',
         
        'BeeColonies-08-01-2019/hcny_p08_t002.csv', 'BeeColonies-08-01-2019/hcny_p09_t013.csv',
        'BeeColonies-08-01-2019/hcny_p10_t009.csv', 'BeeColonies-08-01-2019/hcny_p11_t010.csv',
        'BeeColonies-08-01-2019/hcny_p12_t012.csv']

In [ ]:
colonies = pd.concat([create_dataframe(folder + file, index=0) for file in files], axis=0)
colonies = colonies.reset_index(drop=True)
colonies.replace(['(X)', '(Z)', '-'], 0, inplace=True)
colonies.iloc[:, 1:-1] = colonies.iloc[:, 1:-1].astype(int)
colonies

In [ ]:
colonies.loc[colonies.State == 'United States']

Although the data provided is very recent it is only for the interval 2015-2019

In [ ]:
column = 'Lost'
state = colonies.loc[colonies.State == 'United States', ['Date', column]]
plt.plot(state.Date, state[column], label=f'Change in number of colonies {column}')
plt.plot(state.Date, [state[column].mean()] * len(state.Date), label=f'Mean number of colonies {column}')
plt.xticks(rotation=45)
plt.title(f'Number of colonies {column}')
plt.xlabel('Date')
plt.ylabel('Number of colonies')
plt.legend();

In [ ]:
disorder = pd.concat([create_dataframe(folder + file, index=1) for file in files2], axis=0)
disorder = disorder.reset_index(drop=True)
disorder.replace(['(X)', '(Z)', '-'], 0, inplace=True)
disorder.iloc[:, 1:-1] = disorder.iloc[:, 1:-1].astype(float)
disorder.loc[disorder.State == 'United States']

In [ ]:
stressor = 'Pesticides'
state = disorder.loc[disorder.State == 'United States', ['Date', stressor]]
plt.plot(state.Date, state[stressor], label='Percentage change during time')
plt.plot(state.Date, [state[stressor].mean()] * len(state.Date), label='Mean percentage')
plt.xticks(rotation=45)
plt.title(f'Percent of colonies affected by {stressor}')
plt.xlabel('Date')
plt.ylabel('Percent')
plt.legend();

Some of the known reasons for the colony losses are mites, parasites, pesticides and diseases. Using the disorder dataframe we can see how percentage of colonies affected by stressors. A colony can be affected by multiple stressors at the same time.

In [ ]:
stressor = 'Parasites'

state_colonies = colonies.loc[colonies.State == 'United States', ['Date', 'Colonies', 'Lost', 'Percent lost', 'Added', 'Renovated', 'Percent renovated']]
state_colonies.set_index('Date', inplace=True)
state_disorder = disorder.loc[disorder.State == 'United States', :]
state_disorder.set_index('Date', inplace=True)

joined = state_colonies.join(state_disorder)
joined.drop('State', axis=1, inplace=True)

In [ ]:
joined[stressor] = joined[stressor]/100 * state_colonies['Colonies']
joined

In [ ]:
joined.corr()

It actually makes sense. You can look at this correlations from another point of view. If you have more colonies, then you could lose more of them and that's why it's negatively correlated. Why are the stressors and colonies lost negatively correlated? Maybe you can look at this this way: you loose colonies because of those stuff and the more you loose affected colonies, the less affected colonies you will have and that's why all of those are negatively correlated. Because you loose more affected colonies and less not affected!

### Government investments

We talked to a specialist in the field working in the Swiss Bee Research Center and he told us that one of the reasons why bee colony numbers decline is because the government stops supporting beekepers. Since in this new dataset we have some data about government investments, we will try to analyze that.

In [ ]:
def read_investments(path, index):
    data = []
    
    with open(path, encoding="utf8", errors='ignore') as input_file:
        reader = csv.reader(input_file)
        
        for cnt, row in enumerate(reader):
                
            if row[1] == 'd':
                data.append(row[2:])
    df = pd.DataFrame(data).T[:3]
    df.columns = df.iloc[0]
    df.drop(0, inplace=True)
    df.index = index
    return df.astype(int)

In [ ]:
investments = pd.concat([read_investments('data/Honey/Hone-03-22-2017/hony_p06a_t017.csv', [2015, 2016]),
                        read_investments('data/Honey/hony0519/hony_p05a_t024.csv', [2017, 2018])])

investments

In [ ]:
mask = colonies['Date'].map(lambda x: x.month) == 3
selected = colonies.loc[(mask) & (colonies.State == 'United States')].iloc[:-1]
selected.index = investments.index
selected

In [ ]:
joined = selected.join(investments)
joined.drop(['Date', 'State', 'Maximum'], axis=1, inplace=True)
joined

In [ ]:
joined.corr()

### Colony loss in Europe

New dataset, again porrly formatted!

In [ ]:
df2016 = pd.read_csv('data/Europe/T0001-10.1080_00218839.2016.1260240.csv')
df2016['Year'] = 2016
df2017 = pd.read_csv('data/Europe/T0001-10.1080_00218839.2018.1460911.csv')
df2017['Year'] = 2017

df2018 = pd.read_csv('data/Europe/t0001-10.1080_00218839.2019.1615661.csv')
df2018['Year'] = 2018

In [ ]:
df2016.columns = df2016.columns.str.lower()
df2017.columns = df2017.columns.str.lower()
df2018.columns = df2018.columns.str.lower()

df2017 = df2017.loc[:, df2016.columns]
df2018 = df2018.loc[:, df2016.columns]

In [ ]:
joined = df2016.append(df2017.append(df2018))
joined = joined.loc[~(joined.country.str.strip() == '')]
joined.replace('\(.*\)', '', regex=True, inplace=True)
joined.replace('\(.*', '', regex=True, inplace=True)
joined.replace(',', '', regex=True, inplace=True)
joined.replace('Overall.*', 'Overall', regex=True, inplace=True)
joined.replace('na', np.NaN, inplace=True)
joined['year'] = pd.to_datetime(joined['year'].astype(str))
joined.iloc[:,1:-3] = joined.iloc[:,1:-3].astype(float)
joined

In [ ]:
joined.corr()

In [ ]:
winter_loss = joined.loc[:, ['country', 'overall winter loss rate (95% ci)', 'year']]

In [ ]:
winter_loss.loc[winter_loss['year'] == '2018'].plot.barh(x='country', y='overall winter loss rate (95% ci)',
                                                        figsize=(15,10));

In [ ]:
def convert(data, mask, year):
    df = data.copy()
    df = df.loc[(df['year'] == year) & df['country'].isin(mask)].T
    df.columns = list(df.iloc[0])
    df.drop(df.index[0], inplace=True)
    df.drop(df.index[1], inplace=True)
    df.index = [year]
    return df

In [ ]:
mask = list(set(list(winter_loss.loc[winter_loss['year'] == '2016', 'country'])) 
            & set(list(winter_loss.loc[winter_loss['year'] == '2017', 'country'])) 
            & set(list(winter_loss.loc[winter_loss['year'] == '2018', 'country'])))

In [ ]:
len(mask)

This data is bad, only 22 countries in the intersection, out of 37 in 2018 data

In [ ]:
countries16 = convert(winter_loss.loc[(winter_loss['year'] == '2016') & winter_loss['country'].isin(mask)], mask, '2016')
countries17 = convert(winter_loss.loc[(winter_loss['year'] == '2017') & winter_loss['country'].isin(mask)], mask, '2017')
countries18 = convert(winter_loss.loc[(winter_loss['year'] == '2018') & winter_loss['country'].isin(mask)], mask, '2018')

In [ ]:
stack = pd.concat([countries16, countries17, countries18], sort=True)
stack

In [ ]:
stack.T.plot.bar(figsize=(10, 8), rot=60);

We can see that this is just a mess and we cannot find any meaningful global trends.

# Looking at the price and production of natural honey

Here we would like to see if we can correlate the price, the production of honey with the number of beehives for each country.

First we get the data:

In [ ]:
#load the data
livestock = pd.read_csv('data/Production_Livestock_E_All_Data_(Normalized).csv',  encoding='iso-8859-1')
bees=livestock.loc[livestock.Item=='Beehives']

prices = pd.read_csv('data/Prices_E_All_Data_(Normalized).csv',  encoding='iso-8859-1')
honey_price=prices.loc[prices.Item == 'Honey, natural']

production=pd.read_csv('data/Production_LivestockPrimary_E_All_Data_(Normalized).csv', encoding='iso-8859-1')
honey_production=production.loc[(production.Item=='Honey, natural')  & (production.Element=='Production')]

In [ ]:
countries = ['Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia','Australia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium','Belgium-Luxembourg', 'Belize', 'Bolivia (Plurinational State of)','Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Burundi','Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile','China', 'Colombia', 'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba','Cyprus', 'Czechia', 'Czechoslovakia', 'Dominican Republic','Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Ethiopia','Ethiopia PDR', 'Fiji', 'Finland', 'France', 'French Polynesia','Georgia', 'Germany', 'Greece', 'Greenland', 'Guadeloupe', 'Guam','Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti','Honduras', 'Hungary', 'India', 'Iran (Islamic Republic of)','Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kenya','Kyrgyzstan', 'Latvia', 'Lebanon', 'Libya', 'Liechtenstein','Lithuania', 'Luxembourg', 'Madagascar', 'Mali', 'Martinique','Mexico', 'Mongolia', 'Montenegro', 'Morocco', 'Mozambique','Myanmar', 'Netherlands', 'New Caledonia', 'New Zealand','Nigeria', 'Niue', 'Occupied Palestinian Territory', 'Oman','Pakistan', 'Paraguay', 'Poland', 'Portugal', 'Puerto Rico','Republic of Korea', 'Republic of Moldova', 'Romania','Russian Federation', 'Rwanda', 'Samoa', 'Senegal', 'Serbia','Serbia and Montenegro', 'Slovakia', 'Slovenia', 'South Africa','Spain', 'Sudan', 'Sudan (former)', 'Sweden', 'Switzerland','Syrian Arab Republic', 'Tajikistan','The former Yugoslav Republic of Macedonia', 'Timor-Leste','Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Tuvalu','Uganda', 'Ukraine', 'United Kingdom','United Republic of Tanzania', 'United States of America','Uruguay', 'USSR', 'Uzbekistan','Venezuela (Bolivarian Republic of)', 'Viet Nam','Wallis and Futuna Islands', 'Yemen', 'Yugoslav SFR', 'Zambia']

## Let's have a look a the data 

For each country, we plot the honey price and production on one figure and the number of beehives in a second one

In [ ]:
@interact
def HoneyPrice(country=countries):
    h_p = honey_price.loc[(honey_price.Area == country) & (honey_price.Unit =='USD')]
    b=bees.loc[bees.Area==country]
    prod=honey_production.loc[(honey_production.Area==country)]
    
    fig, ax1 = plt.subplots(2)
    
    fig.set_size_inches(18, 10)

    color = 'tab:red'
    ax1[0].set_title('Honey price and Production')
    ax1[0].set_xlabel('year')
    ax1[0].set_ylabel('USD price', color=color)
    ax1[0].plot(h_p.Year, h_p.Value, color=color)
    ax1[0].tick_params(axis='y', labelcolor=color)

    ax2 = ax1[0].twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:blue'
    ax2.set_ylabel('Production in ', color=color)  # we already handled the x-label with ax1
    ax2.plot(prod.Year, prod.Value, color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    
    
    ax1[1].plot(b.Year,b.Value)
    ax1[1].set_title('Number of beehives')

We can see here that there is a lot of data missing, especilly for the prices. most of the time we have no price data or only chuncks of it, not even from the same time period.

We can still try to correlate the data we have but we need to make sure that the series we are correlating have the same size and correspond to the same time period, in other case, it would not make any sense.



This utilities functions compute the correlation between the price and production of honey and the number of beehives for a particular country.

They check if correlation is possible and return the pearson's correlation of the extracted series
A Nan is returned if the correlation was not possible.

In [ ]:
#Extract only the countries data
bees_countries = bees[bees['Area'].isin(countries)]
price_countries = honey_price[honey_price.Area.isin(countries)]
prod_countries = honey_production[honey_production.Area.isin(countries)]

In [ ]:
def price_beehives_corr(c):
    bee = bees_countries.loc[bees_countries.Area == c]
    prod = prod_countries.loc[(prod_countries.Area == c)]
    
    if len(prod)>0 and len(bee)>0:
        min_year = max(bee.Year.min(),prod.Year.min())
        max_year = min(bee.Year.max(),prod.Year.max())

        bee = bee.loc[(bee.Year >= min_year) & (bee.Year<=max_year)]
        prod = prod.loc[(prod.Year >= min_year) & (prod.Year<=max_year)]
        
        
        if len(prod) == len(bee) and len(bee)  > 9: #we check only if we have at least tenyears of data
            try:
                return sc.stats.pearsonr(bee.Value,prod.Value)[0]
            except:
                return np.nan
        else:
            return np.nan
    else:
        return np.nan

In [ ]:
def production_beehives_corr(c):
    bee = bees_countries.loc[bees_countries.Area == c]
    price = price_countries.loc[(price_countries.Area == c) & (price_countries.Unit == 'USD')]
    
    if len(price)>0 and len(bee)>0:
        min_year = max(bee.Year.min(),price.Year.min())
        max_year = min(bee.Year.max(),price.Year.max())

        bee = bee.loc[(bee.Year >= min_year) & (bee.Year<=max_year)]
        price = price.loc[(price.Year >= min_year) & (price.Year<=max_year)]
        
        
        if len(price) == len(bee) and len(bee)  > 9: #we check only if we have at least tenyears of data
            try:
                return sc.stats.pearsonr(bee.Value,price.Value)[0]
            except:
                return np.nan
        else:
            return np.nan
    else:
        return np.nan

In [ ]:
def price_production_corr(c):
    prod = prod_countries.loc[prod_countries.Area == c]
    price = price_countries.loc[(price_countries.Area == c) & (price_countries.Unit == 'USD')]
    
    if len(price)>0 and len(prod)>0:
        min_year = max(prod.Year.min(),price.Year.min())
        max_year = min(prod.Year.max(),price.Year.max())

        prod = prod.loc[(prod.Year >= min_year) & (prod.Year<=max_year)]
        price = price.loc[(price.Year >= min_year) & (price.Year<=max_year)]
        
        
        if len(price) == len(prod) and len(prod)  > 9: #we check only if we have at least tenyears of data
            try:
                return sc.stats.pearsonr(prod.Value,price.Value)[0]
            except:
                return np.nan
        else:
            return np.nan
    else:
        return np.nan

Now we can compute each of these correlation in a new dataFrame:

In [ ]:
corrs = []
index=[]
for c in countries:
    corr = []
    corr.append(price_beehives_corr(c))
    corr.append(price_production_corr(c))
    corr.append(production_beehives_corr(c))
    
    corrs.append(corr)
    index.append(c)
    
corrs = pd.DataFrame(data=corrs, columns=['price-beehives','price-prod','prod-beehives'],index=index)  

In [ ]:
corrs.head(20)

We can already see that a lot of data is missing.

Let's plot the distribution of the correlations 

In [ ]:
plt.figure(figsize=[20,20])
plt.subplot(221)
plt.hist(corrs['price-beehives'], bins=20)
plt.title('Distribution of the price and \n the number of beehives correlation\n accross countries ')

plt.subplot(222)
plt.hist(corrs['price-prod'], bins=20)
plt.title('Distribution of the price and \n the honey production correlation\n accross countries ')

plt.subplot(223)
plt.hist(corrs['prod-beehives'], bins=20)
plt.title('Distribution of the number of beehives and \n the honey production correlation\n accross countries ')

The distribution of the correlation of the honey price with the number of beehives seems to be left skewed, We can also distinguish the same pattern on the price/production plot. This could mean that, world wide, the number of beehives is generally dependent on the price of the honey.

If we find a way to predict the price of honey in the next year, we could be able to deduce if the number of beehives is going to increase or decrease in the next years.

This needs further research because too much data is missing, it's hard to say if those result are justified.

# Crops and beehives
The bees are the most important insects in the pollination process and we read that the pollination market is much bigger than the honey market. We will try to find some correlations between the yield of crop production and the number of beehives.

In [ ]:
livestock = pd.read_csv('data/Production_Livestock_E_All_Data_(Normalized).csv', encoding='iso-8859-1')
beehives = livestock.loc[livestock['Item'] == 'Beehives', ['Area', 'Value', 'Year']]
beehives.columns = ['Area', 'Beehives', 'Year']
beehives.reset_index(drop=True, inplace=True)
beehives.head()

In [ ]:
crops = pd.read_csv('data/FAOSTAT/Production_Crops_E_All_Data_(Normalized).csv', encoding='iso-8859-1')
crops = crops.loc[(crops['Element'] == 'Yield'), ['Area', 'Item', 'Year', 'Value']]
crops.reset_index(drop=True, inplace=True)
crops.sort_values(inplace=True, by=['Area', 'Item', 'Year'])
crops.columns = ['Area', 'Item', 'Year', 'Tonnes']
crops.head()

In [ ]:
joined = crops.merge(beehives, on=['Area', 'Year'])
joined

In [ ]:
run_this_cell = False

if run_this_cell:
    
    crops_unique = crops['Item'].unique()
    countries_unique = crops['Area'].unique()

    countries_list = []
    crops_list = []
    corrs = []

    for country in countries_unique:
        for crop in crops_unique:

            corr = joined.loc[(joined['Area'] == country) & (joined['Item'] == crop), 
                              ['Tonnes', 'Beehives']].corr().iloc[0, 1]

            countries_list.append(country)
            crops_list.append(crop)
            corrs.append(corr)

    correlations = pd.DataFrame({'country': countries_list, 'crop': crops_list, 'correlation': corrs})
    correlations.to_pickle('crops_correlations')

In [ ]:
correlations = pd.read_pickle('crops_correlations')

We read that one of the crops for which the bees are very important are almonds, that's why we will look into correlations between almonds production per area and number of beehives for all the continents and the World.

In [ ]:
correlations.loc[(correlations['Crop'] == 'Almonds, with shell')]

In [ ]:
continents = ['Africa', 'Europe', 'Asia', 'Central America', 'South America', 'Australia']

cont_corr = correlations.loc[(correlations['Crop'] == 'Almonds, with shell') 
                 & (correlations['Area'].isin(continents)), ['Area', 'Correlation']]

cont_corr.plot.bar(x='Area', y='Correlation', legend=False, rot=45);
plt.title('Pearson correlation of beehives number and almonds yield (harvested production per unit of harvested area)');

We can see that there are some very big correlations between those. It would make sense to this for other crops as well.

In [ ]:
correlations.loc[(correlations['Area'] == 'World')].plot.barh(x='Crop', y='Correlation', figsize=(10, 30));

Wow!

# Conclusion

In this notebook we provide a descriptive analysis of the beehives dataset. Other datasets have also been analyzed in order to find correlations with the beehives data. The main discovery of this phase is that, against all our expectations, the number of beehives on the planet is increasing! This is good news! However, even if the global trend is increasing, in some countries the beehives population is decreasing. 

In the analysis we have then tried to find what factors could have an impact on this. In particular the factors taken into consideration are: 
- bees colonies stressers(parasites, diseases): datasets for USA and Europe only (interesting for knowledge about the topic, but not enough data to correlate to the FAO beehives dataset)
- pesticides: dataset not rich enough to find real correlations
- forests land: interesting correlation
- inland waters: interesting correlation
- agricultural land: interesting correlation
- honey price: dataset not rich enough
- government investments: not enough data(2015-2019 and just for USA)

Our priority for the next step will be to focus on which of the so far collected information will actually be useful and necessary to build a data story. We will continue our research in this direction, following those factors that at the moment seem to be the more related to beehives population. 
In this regard, we need to improve some of the metrics used in the analysis, such as the ratio of beehives (now divided by total land country surface) and the measure of increasing/decreasing of beehives population over the years.

Another step will be to use the most promising factors to find a model that is able to predict the number of beehives. We don't really know if this is feasible or not, but it would be nice to be able to determine the beehives population given some other external factors.

Finally, for the last step we are going to summarize all the collected knowledge into a well structured data story, and after that present data in visually appealing plots and maps into a website.

# A list of internal milestones up until project milestone 3
### 25. November (1st week)
- Making the maps interactive
- Building the website skeletton

### 2. December (2nd week)
- Selecting relevant parts for the datastory
- Work on feedback
- Create the story 

### 9. December (3rd week)
- Continue on the story 
- Putting the story on the webpage
- Clean the notebook so only relevant parts appear

### 16. December (4th week)
- Resolve issues on the page
- Beautify the page 
- Clean code 

### 20. December (Deadline)
